# 5. Use CML API to Define a Job Which Populates Chroma Vector DB
The benefit to using the CML API to create a job is that the user can then take a programmatic approach to creating jobs and then running them. 
Using the cmlapi library to create jobs is beneficial because it enables automation, version control, reproducibility, integration, scalability, error handling, and efficiency in job management, streamlining data processing workflows. 

#### 5.1 Declare imports, create CML API client, and list available runtimes
Imports necessary modules, define a collection name, initialize a CML client, and retrieve a list of available runtimes that match specific criteria, printing the list of available runtimes.

In [63]:
import os
import cmlapi
import random
import string
import json

COLLECTION_NAME = 'cml-default' ## Update if you have changed this
    
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
available_runtimes = client.list_runtimes(search_filter=json.dumps({
    "kernel": "Python 3.10",
    "edition": "Nvidia GPU",
    "editor": "JupyterLab"
}))
print(available_runtimes)

#### 5.2 Retrieve the latest ML Runtime Identifier and save to an environment variable

In [ ]:
## Set available runtimes to the latest runtime in the environment (iterator is the number that begins with 0 and advances sequentially)
## The JOB_IMAGE_ML_RUNTIME variable stores the ML Runtime which will be used to launch the job
print(available_runtimes.runtimes[1])
print(available_runtimes.runtimes[1].image_identifier)
JOB_IMAGE_ML_RUNTIME = available_runtimes.runtimes[1].image_identifier

## Store the ML Runtime for any future jobs in an environment variable so we don't have to do this step again
os.environ['JOB_IMAGE_ML_RUNTIME'] = JOB_IMAGE_ML_RUNTIME

#### 5.3 Get the current working project
Get and print the metadata of the current working project

In [65]:
project = client.get_project(project_id=os.getenv("CDSW_PROJECT_ID"))
print(project)

{'created_at': datetime.datetime(2023, 12, 11, 17, 36, 11, 602406, tzinfo=tzlocal()),
 'creation_status': 'success',
 'creator': {'email': 'ktalbert@cloudera.com',
             'name': 'Kevin Talbert',
             'username': 'ktalbert'},
 'default_engine_type': 'ml_runtime',
 'description': '',
 'environment': '{"CDSW_APP_POLLING_ENDPOINT":"/","PROJECT_OWNER":"ktalbert"}',
 'ephemeral_storage_limit': 10,
 'ephemeral_storage_request': 0,
 'id': 'i7c6-5g66-06pw-iu5i',
 'name': 'CML-LLM-HOL-Workshop',
 'owner': {'email': 'ktalbert@cloudera.com',
           'name': 'Kevin Talbert',
           'username': 'ktalbert'},
 'permissions': {'admin': True,
                 'business_user': True,
                 'inherit': False,
                 'operator': True,
                 'read': True,
                 'write': True},
 'shared_memory_limit': 0,
 'updated_at': datetime.datetime(2023, 12, 11, 18, 27, 54, 4162, tzinfo=tzlocal()),
 'visibility': 'private'}


#### 5.4 Create and Run Job to Populate Chroma Vector DB

This code generates a random identifier, creates a job request for populating a Chroma Vector database with specified parameters such as project ID, script, and resource allocation, and then creates the job and a corresponding job run within the Cloudera Machine Learning environment, effectively initiating a task to populate the vector DB.

In [66]:
random_id=''.join(random.choice(string.ascii_lowercase) for i in range(10))
job_body = cmlapi.CreateJobRequest(
    project_id = project.id,
    name = "Populate Chroma Vector DB " + random_id, 
    kernel = "python3", 
    script = "5_populate_local_chroma_db/populate_chroma_vectors.py",
    cpu = 1,
    memory = 4,
    runtime_identifier = os.getenv('JOB_IMAGE_ML_RUNTIME')
)

job_result = client.create_job(
    body = job_body, 
    project_id = str(project.id)
)

job_run = client.create_job_run(
    cmlapi.CreateJobRunRequest(),
    project_id = project.id, 
    job_id = job_result.id
)